# 上采样方式1：反卷积，转置卷积
* 参考动态图：https://github.com/vdumoulin/conv_arithmetic
* 下面是反卷积的效果图
* 论文是：《Learning Deconvolution Network for Semantic Segmentation》
<div style="text-align: center">
    <img src="ConvTranspose1.png"/>
</div>

* (a)是输入的原图，(b)到(j)是输出的底级别到高级别的featuremap
* (b)是最后的14x14 deconv
* (c)是28x28的unpooling layer
* (d)是28x28的deconv
* (e)是56x56的unpooling
* (f)是56x56的deconv
* (g)是112x112的unpooling
* (h)是112x112的deconv
* (i)是224x224的unpooling
* (j)是224x224的deconv

* 输出尺寸计算：
- Input: $(N, C_{in}, H_{in}, W_{in})$
- Output: $(N, C_{out}, H_{out}, W_{out})$

$$
      H_{out} = (H_{in} - 1) \times \text{stride}[0] - 2 \times \text{padding}[0] + \text{dilation}[0]
                \times (\text{kernel_size}[0] - 1) + \text{output_padding}[0] + 1
$$
$$
      W_{out} = (W_{in} - 1) \times \text{stride}[1] - 2 \times \text{padding}[1] + \text{dilation}[1]
                \times (\text{kernel_size}[1] - 1) + \text{output_padding}[1] + 1
$$

* 反卷积的计算方法，stride=1，kernel_size=3，无padding：
<div style="text-align: center">
    <img src="反卷积演示.png"/>
</div>

* 反卷积的计算方法，stride、padding、 dilation都有的样子：
<div style="text-align: center">
    <img src="ConvTranspose通用朴适.png"/>
</div>

* 反卷积论文示意图
<div style="text-align: center">
    <img src="ConvTranspose2.png"/>
</div>
<div style="text-align: center">
    <img src="ConvTranspose3.png"/>
</div>


# ConvTranspose2d，转置卷积，反卷积，torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
1. 反卷积，实则是对输入做了变换后的卷积操作。并不等价于可以逆变换卷积

In [1]:
import torch
import torch.nn as nn

In [27]:
transpose = nn.ConvTranspose2d(1, 1, kernel_size=3, stride=1, padding=0, dilation=1, output_padding=0)
transpose.weight.data = torch.eye(3, 3).view(1, 1, 3, 3)
transpose.bias.data.fill_(0)
transpose(torch.ones(1, 1, 2, 2))

tensor([[[[1., 1., 0., 0.],
          [1., 2., 1., 0.],
          [0., 1., 2., 1.],
          [0., 0., 1., 1.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [28]:
transpose = nn.ConvTranspose2d(1, 1, kernel_size=3, stride=2, padding=1, dilation=2, output_padding=0)
transpose.weight.data = torch.eye(3, 3).view(1, 1, 3, 3)
transpose.bias.data.fill_(0)
transpose(torch.tensor([[1, 2], [3, 4.]]).view(1, 1, 2, 2))

tensor([[[[0., 0., 0., 0., 0.],
          [0., 5., 0., 2., 0.],
          [0., 0., 0., 0., 0.],
          [0., 3., 0., 5., 0.],
          [0., 0., 0., 0., 0.]]]], grad_fn=<SlowConvTranspose2DBackward>)

# 上采样方式2，torch.nn.Upsample(size=None, scale_factor=None, mode='nearest', align_corners=None)
* size，输出的大小
* scale_factor，对于输入的缩放系数
* mode，插值方式，可选为：'nearest', 'linear', 'bilinear', 'bicubic', 'trilinear'. 默认: 'nearest'
* align_corners，对齐边角，只在mode为'linear', 'bilinear',或者 'trilinear'时有效，默认False
* 注意：不可以同时制定size和scale_factor，否则这是不明确的行为

In [30]:
x = torch.arange(4).view(1, 1, 2, 2).float()
up = nn.Upsample(scale_factor=2, mode="nearest")
up(x)

tensor([[[[0., 0., 1., 1.],
          [0., 0., 1., 1.],
          [2., 2., 3., 3.],
          [2., 2., 3., 3.]]]])

In [31]:
x = torch.arange(4).view(1, 1, 2, 2).float()
up = nn.Upsample(scale_factor=2, mode="bilinear")
up(x)

tensor([[[[0.0000, 0.2500, 0.7500, 1.0000],
          [0.5000, 0.7500, 1.2500, 1.5000],
          [1.5000, 1.7500, 2.2500, 2.5000],
          [2.0000, 2.2500, 2.7500, 3.0000]]]])

In [20]:
x = torch.arange(4).view(1, 1, 2, 2).float()
x

tensor([[[[0., 1.],
          [2., 3.]]]])

In [21]:
up = nn.Upsample(scale_factor=2, mode="nearest")
up(x)

tensor([[[[0., 0., 1., 1.],
          [0., 0., 1., 1.],
          [2., 2., 3., 3.],
          [2., 2., 3., 3.]]]])

In [24]:
up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)

# 不对齐边角时，y方向差值为：0.5, 1, 0.5
up(x)

tensor([[[[0.0000, 0.2500, 0.7500, 1.0000],
          [0.5000, 0.7500, 1.2500, 1.5000],
          [1.5000, 1.7500, 2.2500, 2.5000],
          [2.0000, 2.2500, 2.7500, 3.0000]]]])

In [26]:
up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)

# 对齐边角时，y方向差值为：0.666, 0.666, 0.666
up(x)

tensor([[[[0.0000, 0.3333, 0.6667, 1.0000],
          [0.6667, 1.0000, 1.3333, 1.6667],
          [1.3333, 1.6667, 2.0000, 2.3333],
          [2.0000, 2.3333, 2.6667, 3.0000]]]])

# 上采样方法（nn.PixelShuffle）
* upscale_factor：指定为需要上采样的倍数
* 输入维度为：$( N, L, H_{in}, W_{in} )$ 这里 $(L = C * upscale\_factor^2)$
* 输出维度为：$( N, C, H_{out}, W_{out} )$
    - $H_{out} = H_{in} \times \text{upscale_factor}$
    - $W_{out} = W_{in} \times \text{upscale_factor}$

<img src="pixshuffle.png"/>

- 多用在分割任务上